In [1]:
import datetime
import ee
import hydrafloods as hf

ee.Initialize()


In [2]:
from IPython.display import Image

## Setting up a process

In [3]:
# define region
region = ee.Geometry.Point(105,12).buffer(25000).bounds()

In [4]:
# define time period
startTime = datetime.datetime(2019,9,15)
endTime = startTime + datetime.timedelta(5)

In [5]:
# get a collection of images based on region and time
s1 = hf.collection.Sentinel1(region,startTime,endTime)

s1

HYDRAFloods Collection:
{'assetid': 'COPERNICUS/S1_GRD',
 'endTime': '2019-09-20',
 'name': 'Sentinel1',
 'region': [[[...], [...], [...], [...], [...]]],
 'startTime': '2019-09-15'}

In [6]:
print(f"Number of images: {s1.nImages}")

Number of images: 3


Here is a little code to display the first image in the collection to visualize what is happening.

In [7]:
# reduce the collection to mean values (mosaic), then get a thumbnail view
thumbnail = ee.Image(s1.collection.first()).getThumbUrl({'region':region,'min':-25,'max':0,'bands':'VV','format':'png','dimensions':512})
# display result
Image(url = thumbnail)

## Apply some preprocessing
In this case, we will apply a speckle filter on all of the images.

In [8]:
# apply the lee sigma speckle filter
filtered = s1.apply(hf.filtering.refinedLee)

In [9]:
# get the filtered collection for display
thumbnail = ee.Image(filtered.collection.first()).getThumbUrl({'region':region,'min':-25,'max':0,'bands':'VV','format':'png','dimensions':512})
Image(url = thumbnail)

## Apply a water mapping algorithm

In [10]:
# apply a water thresholding algorithm to the collection
water = filtered.apply(hf.thresholding.bmaxOtsu,initialThreshold=-16)

In [11]:
# view results
thumbnail = ee.Image(water.collection.first()).getThumbUrl({'min':0,'max':1,'palette':'lightgray,darkblue','region':region,'dimensions':512})
Image(url = thumbnail)

### Export the results

In [12]:
hf.geeutils.batchExport(water.collection,'users/kelmarkert/hydrafloods_examples',
                        prefix='Sentinel1',suffix='bmax',scale=10,
                        metadata={'sensor':'sentinel1','algorithm':'bmax'},
                        pyramiding={'.default':'mode'},
                        verbose=True)

running export for Sentinel1_20190917_bmax
running export for Sentinel1_20190916_bmax
running export for Sentinel1_20190915_bmax


In [13]:
ee.batch.Task.list()[:water.nImages]

[<Task EXPORT_IMAGE: Sentinel1_20190915_bmax (READY)>,
 <Task EXPORT_IMAGE: Sentinel1_20190916_bmax (READY)>,
 <Task EXPORT_IMAGE: Sentinel1_20190917_bmax (RUNNING)>]